# Задание 2.1 - Нейронные сети

В этом задании вы реализуете и натренируете настоящую нейроную сеть своими руками!

В некотором смысле это будет расширением прошлого задания - нам нужно просто составить несколько линейных классификаторов вместе!

<img src="https://i.redd.it/n9fgba8b0qr01.png" alt="Stack_more_layers" width="400px"/>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
from dataset import load_svhn, random_split_train_val
from gradient_check import check_layer_gradient, check_layer_param_gradient, check_model_gradient
from layers import FullyConnectedLayer, ReLULayer
from model import TwoLayerNet
from trainer import Trainer, Dataset
from optim import SGD, MomentumSGD
from metrics import multiclass_accuracy

# Загружаем данные

И разделяем их на training и validation.

In [3]:
def prepare_for_neural_network(train_X, test_X):
    train_flat = train_X.reshape(train_X.shape[0], -1).astype(float) / 255.0
    test_flat = test_X.reshape(test_X.shape[0], -1).astype(float) / 255.0
    
    # Subtract mean
    mean_image = np.mean(train_flat, axis = 0)
    train_flat -= mean_image
    test_flat -= mean_image
    
    return train_flat, test_flat
    
train_X, train_y, test_X, test_y = load_svhn("data", max_train=10000, max_test=1000)    
train_X, test_X = prepare_for_neural_network(train_X, test_X)
# Split train into train and val
train_X, train_y, val_X, val_y = random_split_train_val(train_X, train_y, num_val = 1000)

# Как всегда, начинаем с кирпичиков

Мы будем реализовывать необходимые нам слои по очереди. Каждый слой должен реализовать:
- прямой проход (forward pass), который генерирует выход слоя по входу и запоминает необходимые данные
- обратный проход (backward pass), который получает градиент по выходу слоя и вычисляет градиент по входу и по параметрам

Начнем с ReLU, у которого параметров нет.

In [4]:
# TODO: Implement ReLULayer layer in layers.py
# Note: you'll need to copy implementation of the gradient_check function from the previous assignment

X = np.array([[1,-2,3],
              [-1, 2, 0.1]
              ])

assert check_layer_gradient(ReLULayer(), X)

Gradient check passed!


А теперь реализуем полносвязный слой (fully connected layer), у которого будет два массива параметров: W (weights) и B (bias).

Все параметры наши слои будут использовать для параметров специальный класс `Param`, в котором будут храниться значения параметров и градиенты этих параметров, вычисляемые во время обратного прохода.

Это даст возможность аккумулировать (суммировать) градиенты из разных частей функции потерь, например, из cross-entropy loss и regularization loss.

In [5]:
# TODO: Implement FullyConnected layer forward and backward methods
assert check_layer_gradient(FullyConnectedLayer(3, 4), X)
# TODO: Implement storing gradients for W and B
assert check_layer_param_gradient(FullyConnectedLayer(3, 4), X, 'W')
assert check_layer_param_gradient(FullyConnectedLayer(3, 4), X, 'B')

Gradient check passed!
Gradient check passed!
Gradient check passed!


## Создаем нейронную сеть

Теперь мы реализуем простейшую нейронную сеть с двумя полносвязным слоями и нелинейностью ReLU. Реализуйте функцию `compute_loss_and_gradients`, она должна запустить прямой и обратный проход через оба слоя для вычисления градиентов.

Не забудьте реализовать очистку градиентов в начале функции.

In [6]:
# TODO: In model.py, implement compute_loss_and_gradients function
model = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 3, reg = 0)
loss = model.compute_loss_and_gradients(train_X[:2], train_y[:2])

# TODO Now implement backward pass and aggregate all of the params
check_model_gradient(model, train_X[:2], train_y[:2])

Checking gradient for (0, 'W')
Gradient check passed!
Checking gradient for (0, 'B')
Gradient check passed!
Checking gradient for (2, 'W')
Gradient check passed!
Checking gradient for (2, 'B')
Gradient check passed!


True

In [7]:
nums_train = train_X.shape[0]
shuffled_indices = np.arange(nums_train)
np.random.shuffle(shuffled_indices)
train_len = int(0.7 * (nums_train))
X_train = train_X[shuffled_indices[:train_len]]
y_train = train_y[shuffled_indices[:train_len]]
X_val = train_X[shuffled_indices[train_len:]]
y_val = train_y[shuffled_indices[train_len:]]

Теперь добавьте к модели регуляризацию - она должна прибавляться к loss и делать свой вклад в градиенты.

In [8]:
# TODO Now implement l2 regularization in the forward and backward pass
model_with_reg = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 3, reg = 1e1)
loss_with_reg = model_with_reg.compute_loss_and_gradients(train_X[:2], train_y[:2])
assert loss_with_reg > loss and not np.isclose(loss_with_reg, loss), \
    "Loss with regularization (%2.4f) should be higher than without it (%2.4f)!" % (loss, loss_with_reg)

check_model_gradient(model_with_reg, train_X[:2], train_y[:2])

Checking gradient for (0, 'W')
Gradient check passed!
Checking gradient for (0, 'B')
Gradient check passed!
Checking gradient for (2, 'W')
Gradient check passed!
Checking gradient for (2, 'B')
Gradient check passed!


True

Также реализуем функцию предсказания (вычисления значения) модели на новых данных.

Какое значение точности мы ожидаем увидеть до начала тренировки?

In [9]:
model_with_reg.predict(train_X[:30])[0]

7

In [10]:
# Finally, implement predict function!

# TODO: Implement predict function
# What would be the value we expect?
multiclass_accuracy(model_with_reg.predict(train_X[:30]), train_y[:30]) 

0.1

# Допишем код для процесса тренировки

Если все реализовано корректно, значение функции ошибки должно уменьшаться с каждой эпохой, пусть и медленно. Не беспокойтесь пока про validation accuracy.

In [11]:
model = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 100, reg = 1e1)
dataset = Dataset(train_X, train_y, val_X, val_y)
trainer = Trainer(model, dataset, SGD(), learning_rate = 1e-2)

# TODO Implement missing pieces in Trainer.fit function
# You should expect loss to go down every epoch, even if it's slow


In [12]:
# loss_history, train_history, val_history = trainer.fit()

In [13]:
# plt.plot(train_history)
# plt.plot(val_history)

# Улучшаем процесс тренировки

Мы реализуем несколько ключевых оптимизаций, необходимых для тренировки современных нейросетей.

## Уменьшение скорости обучения (learning rate decay)

Одна из необходимых оптимизаций во время тренировки нейронных сетей - постепенное уменьшение скорости обучения по мере тренировки.

Один из стандартных методов - уменьшение скорости обучения (learning rate) каждые N эпох на коэффициент d (часто называемый decay). Значения N и d, как всегда, являются гиперпараметрами и должны подбираться на основе эффективности на проверочных данных (validation data). 

В нашем случае N будет равным 1.

In [14]:
# TODO Implement learning rate decay inside Trainer.fit method
# Decay should happen once per epoch

# model = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 100, reg = 1e-1)
# dataset = Dataset(train_X, train_y, val_X, val_y)
# trainer = Trainer(model, dataset, SGD(), learning_rate_decay=0.99)

# initial_learning_rate = trainer.learning_rate
# loss_history, train_history, val_history = trainer.fit()

# assert trainer.learning_rate < initial_learning_rate, "Learning rate should've been reduced"
# assert trainer.learning_rate > 0.5*initial_learning_rate, "Learning rate shouldn'tve been reduced that much!"

# Накопление импульса (Momentum SGD)

Другой большой класс оптимизаций - использование более эффективных методов градиентного спуска. Мы реализуем один из них - накопление импульса (Momentum SGD).

Этот метод хранит скорость движения, использует градиент для ее изменения на каждом шаге, и изменяет веса пропорционально значению скорости.
(Физическая аналогия: Вместо скорости градиенты теперь будут задавать ускорение, но будет присутствовать сила трения.)

```
velocity = momentum * velocity - learning_rate * gradient 
w = w + velocity
```

`momentum` здесь коэффициент затухания, который тоже является гиперпараметром (к счастью, для него часто есть хорошее значение по умолчанию, типичный диапазон -- 0.8-0.99).

Несколько полезных ссылок, где метод разбирается более подробно:  
http://cs231n.github.io/neural-networks-3/#sgd  
https://distill.pub/2017/momentum/

In [15]:
# TODO: Implement MomentumSGD.update function in optim.py

model = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 100, reg = 1e-1)
dataset = Dataset(train_X, train_y, val_X, val_y)
trainer = Trainer(model, dataset, MomentumSGD(), learning_rate=1e-4, learning_rate_decay=0.99)

# You should see even better results than before!
loss_history, train_history, val_history = trainer.fit()

Loss: 42.241475, Train accuracy: 0.196667, val accuracy: 0.206000
Loss: 44.886053, Train accuracy: 0.196667, val accuracy: 0.206000
Loss: 42.114705, Train accuracy: 0.196667, val accuracy: 0.206000
Loss: 43.362908, Train accuracy: 0.196667, val accuracy: 0.206000
Loss: 38.689303, Train accuracy: 0.206333, val accuracy: 0.219000
Loss: 39.848849, Train accuracy: 0.250556, val accuracy: 0.252000
Loss: 43.589987, Train accuracy: 0.275667, val accuracy: 0.277000
Loss: 41.658010, Train accuracy: 0.290778, val accuracy: 0.285000
Loss: 40.675572, Train accuracy: 0.345444, val accuracy: 0.349000
Loss: 38.559594, Train accuracy: 0.399667, val accuracy: 0.390000
Loss: 35.180066, Train accuracy: 0.433556, val accuracy: 0.428000
Loss: 29.114179, Train accuracy: 0.470444, val accuracy: 0.462000
Loss: 31.537373, Train accuracy: 0.498778, val accuracy: 0.499000
Loss: 32.718789, Train accuracy: 0.520000, val accuracy: 0.527000
Loss: 33.333823, Train accuracy: 0.548667, val accuracy: 0.548000
Loss: 33.6

# Ну что, давайте уже тренировать сеть!

## Последний тест - переобучимся (overfit) на маленьком наборе данных

Хороший способ проверить, все ли реализовано корректно - переобучить сеть на маленьком наборе данных.  
Наша модель обладает достаточной мощностью, чтобы приблизить маленький набор данных идеально, поэтому мы ожидаем, что на нем мы быстро дойдем до 100% точности на тренировочном наборе. 

Если этого не происходит, то где-то была допущена ошибка!

In [16]:
data_size = 15
model = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 100, reg = 1e-1)
dataset = Dataset(train_X[:data_size], train_y[:data_size], val_X[:data_size], val_y[:data_size])
trainer = Trainer(model, dataset, SGD(), learning_rate=1e-1, num_epochs=150, batch_size=5)

# You should expect this to reach 1.0 training accuracy 
loss_history, train_history, val_history = trainer.fit()

Loss: 11.449705, Train accuracy: 0.200000, val accuracy: 0.133333
Loss: 11.567443, Train accuracy: 0.200000, val accuracy: 0.133333
Loss: 12.760513, Train accuracy: 0.333333, val accuracy: 0.066667
Loss: 12.086189, Train accuracy: 0.200000, val accuracy: 0.133333
Loss: 20.466829, Train accuracy: 0.200000, val accuracy: 0.133333
Loss: 25.126151, Train accuracy: 0.266667, val accuracy: 0.066667
Loss: 43.138005, Train accuracy: 0.200000, val accuracy: 0.133333
Loss: 34.129820, Train accuracy: 0.200000, val accuracy: 0.000000
Loss: 37.706869, Train accuracy: 0.200000, val accuracy: 0.200000
Loss: 354.156138, Train accuracy: 0.133333, val accuracy: 0.133333
Loss: 1113.769120, Train accuracy: 0.133333, val accuracy: 0.133333
Loss: 1116.027848, Train accuracy: 0.133333, val accuracy: 0.133333
Loss: inf, Train accuracy: 0.200000, val accuracy: 0.066667
Loss: inf, Train accuracy: 0.066667, val accuracy: 0.066667
Loss: inf, Train accuracy: 0.266667, val accuracy: 0.000000
Loss: inf, Train accura

/Users/ildarnasrtdinov/Work/education/dlcourse/assignments/assignment2/layers.py:42: RuntimeWarning: divide by zero encountered in log
  ans = -np.sum(np.log(probs[np.arange(len(target_index)), target_index.reshape(1, -1)]))


Loss: inf, Train accuracy: 0.266667, val accuracy: 0.066667
Loss: inf, Train accuracy: 0.133333, val accuracy: 0.200000
Loss: inf, Train accuracy: 0.200000, val accuracy: 0.066667
Loss: inf, Train accuracy: 0.200000, val accuracy: 0.066667
Loss: inf, Train accuracy: 0.266667, val accuracy: 0.133333
Loss: inf, Train accuracy: 0.133333, val accuracy: 0.200000
Loss: inf, Train accuracy: 0.000000, val accuracy: 0.133333
Loss: inf, Train accuracy: 0.133333, val accuracy: 0.133333
Loss: inf, Train accuracy: 0.000000, val accuracy: 0.133333
Loss: inf, Train accuracy: 0.133333, val accuracy: 0.200000
Loss: inf, Train accuracy: 0.133333, val accuracy: 0.200000
Loss: inf, Train accuracy: 0.066667, val accuracy: 0.066667
Loss: inf, Train accuracy: 0.066667, val accuracy: 0.000000
Loss: inf, Train accuracy: 0.400000, val accuracy: 0.066667
Loss: inf, Train accuracy: 0.133333, val accuracy: 0.200000
Loss: inf, Train accuracy: 0.200000, val accuracy: 0.200000
Loss: inf, Train accuracy: 0.133333, val

Теперь найдем гипепараметры, для которых этот процесс сходится быстрее.
Если все реализовано корректно, то существуют параметры, при которых процесс сходится в **20** эпох или еще быстрее.
Найдите их!

In [17]:
# Now, tweak some hyper parameters and make it train to 1.0 accuracy in 20 epochs or less

model = TwoLayerNet(n_input = train_X.shape[1], n_output = 10, hidden_layer_size = 100, reg = 1e-1)
dataset = Dataset(train_X[:data_size], train_y[:data_size], val_X[:data_size], val_y[:data_size])
# TODO: Change any hyperparamers or optimizators to reach training accuracy in 20 epochs
trainer = Trainer(model, dataset, SGD(), learning_rate=1e-1, num_epochs=20, batch_size=5)

loss_history, train_history, val_history = trainer.fit()

Loss: 11.864046, Train accuracy: 0.200000, val accuracy: 0.066667
Loss: 11.128917, Train accuracy: 0.333333, val accuracy: 0.066667
Loss: 9.940287, Train accuracy: 0.400000, val accuracy: 0.000000
Loss: 9.825753, Train accuracy: 0.266667, val accuracy: 0.066667
Loss: 13.605160, Train accuracy: 0.266667, val accuracy: 0.133333
Loss: 9.744032, Train accuracy: 0.333333, val accuracy: 0.133333
Loss: 12.872586, Train accuracy: 0.266667, val accuracy: 0.066667
Loss: 15.841548, Train accuracy: 0.333333, val accuracy: 0.066667
Loss: 10.804288, Train accuracy: 0.533333, val accuracy: 0.066667
Loss: 14.496527, Train accuracy: 0.200000, val accuracy: 0.133333
Loss: 9.089495, Train accuracy: 0.333333, val accuracy: 0.133333
Loss: 284.363480, Train accuracy: 0.200000, val accuracy: 0.133333
Loss: 1678.967547, Train accuracy: 0.200000, val accuracy: 0.066667
Loss: 1180.122249, Train accuracy: 0.400000, val accuracy: 0.000000
Loss: inf, Train accuracy: 0.400000, val accuracy: 0.133333
Loss: inf, Trai

# Итак, основное мероприятие!

Натренируйте лучшую нейросеть! Можно добавлять и изменять параметры, менять количество нейронов в слоях сети и как угодно экспериментировать. 

Добейтесь точности лучше **60%** на validation set.

In [18]:
# Let's train the best one-hidden-layer network we can

learning_rates = 1e-4
reg_strength = 1e-3
learning_rate_decay = 0.999
hidden_layer_size = 128
num_epochs = 200
batch_size = 64

best_classifier = None
best_val_accuracy = None

loss_history = []
train_history = []
val_history = []

# TODO find the best hyperparameters to train the network
# Don't hesitate to add new values to the arrays above, perform experiments, use any tricks you want
# You should expect to get to at least 40% of valudation accuracy
# Save loss/train/history of the best classifier to the variables above

print('best validation accuracy achieved: %f' % best_val_accuracy)

TypeError: must be real number, not NoneType

In [ ]:
plt.figure(figsize=(15, 7))
plt.subplot(211)
plt.title("Loss")
plt.plot(loss_history)
plt.subplot(212)
plt.title("Train/validation accuracy")
plt.plot(train_history)
plt.plot(val_history)

# Как обычно, посмотрим, как наша лучшая модель работает на тестовых данных

In [ ]:
test_pred = best_classifier.predict(test_X)
test_accuracy = multiclass_accuracy(test_pred, test_y)
print('Neural net test set accuracy: %f' % (test_accuracy, ))